In [8]:
import os
os.getcwd()

'/home/ubuntu'

In [9]:
import findspark
findspark.init()
findspark.find()

'/usr/lib/spark'

In [10]:
import os
import sys

os.environ['PYSPARK_PYTHON'] = sys.executable
os.environ['PYSPARK_DRIVER_PYTHON'] = sys.executable
os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages org.apache.spark:spark-streaming-kafka-0-10_2.12:3.0.3,org.apache.spark:spark-sql-kafka-0-10_2.12:3.0.3 pyspark-shell'
import pyspark
import findspark
from pyspark import SparkContext, SparkConf
from pyspark.sql import SQLContext, SparkSession
import pyspark.sql.functions as F

In [4]:
# groupId = org.apache.spark
# artifactId = spark-sql-kafka-0-10_2.12
# version = 3.5.0

In [97]:
spark = (
    SparkSession
        .builder
        .appName("Streaming-kafka")
        .config("spark.dynamicAllocation.enabled", "true")
        .config("spark.executor.cores", "2")
        .config("spark.executor.memory", "2g")
        .config("spark.executor.instances", "3")
        .config("spark.default.parallelism", "24")
        .config("spark.sql.streaming.kafka.useDeprecatedOffsetFetching", "true")
        .getOrCreate()
)
spark

In [108]:
!spark-shell --version

#version kafka 3.5

OpenJDK 64-Bit Server VM warning: Option UseConcMarkSweepGC was deprecated in version 9.0 and will likely be removed in a future release.
SLF4J: Class path contains multiple SLF4J bindings.
SLF4J: Found binding in [jar:file:/usr/lib/spark/jars/slf4j-log4j12-1.7.30.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: Found binding in [jar:file:/usr/lib/hadoop/lib/slf4j-log4j12-1.7.25.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: See http://www.slf4j.org/codes.html#multiple_bindings for an explanation.
SLF4J: Actual binding is of type [org.slf4j.impl.Log4jLoggerFactory]
Welcome to
      ____              __
     / __/__  ___ _____/ /__
    _\ \/ _ \/ _ `/ __/  '_/
   /___/ .__/\_,_/_/ /_/\_\   version 3.0.3
      /_/
                        
Using Scala version 2.12.10, OpenJDK 64-Bit Server VM, 11.0.21
Branch 
Compiled by user  on 2023-11-07T12:42:26Z
Revision 
Url 
Type --help for more information.


In [109]:
spark.stop()

In [98]:
streaming_df = spark.readStream \
    .format("kafka") \
    .option("kafka.bootstrap.servers", "rc1a-sq626k239t359ent.mdb.yandexcloud.net:9091") \
    .option("kafka.security.protocol", "SASL_SSL") \
    .option("kafka.sasl.mechanism", "SCRAM-SHA-256") \
    .option("kafka.sasl.username", "mlops") \
    .option("kafka.sasl.password", "otus-mlops") \
    .option("subscribe", "transactions_sample") \
    .option("kafka.group.id", "mlops") \
    .option("startingOffsets", "earliest") \
    .load()

In [99]:
streaming_df.printSchema()

root
 |-- key: binary (nullable = true)
 |-- value: binary (nullable = true)
 |-- topic: string (nullable = true)
 |-- partition: integer (nullable = true)
 |-- offset: long (nullable = true)
 |-- timestamp: timestamp (nullable = true)
 |-- timestampType: integer (nullable = true)



In [100]:
from pyspark.sql.types import StringType, StructField, StructType, ArrayType, LongType
json_schema = StructType([StructField("ts", StringType(), True), \
                          StructField("user_id", StringType(), True), \
                          StructField("page_id", StringType(), True)])


# json_schema = StructType([StructField('ts', StringType(), True), \
# StructField('user_id', StructType([StructField('page_id', ArrayType(StructType([ \
# StructField('deviceId', StringType(), True), \
# StructField('measure', StringType(), True), \
# StructField('status', StringType(), True), \
# StructField('temperature', LongType(), True)]), True), True)]), True), \
# StructField('eventId', StringType(), True), \
# StructField('eventOffset', LongType(), True), \
# StructField('eventPublisher', StringType(), True), \
# StructField('eventTime', StringType(), True)])

In [101]:
json_df = streaming_df.selectExpr("cast(value as string) as value")

In [ ]:
# json_df.show()

In [102]:
from pyspark.sql.types import MapType,StringType
from pyspark.sql.functions import from_json

json_expanded_df = json_df.withColumn("value", from_json(json_df["value"], json_schema)).select("value.*") 

In [103]:
json_expanded_df

DataFrame[ts: string, user_id: string, page_id: string]

In [104]:
from pyspark.sql.functions import explode, col, split

exploded_df = json_expanded_df \
    .select("ts", "user_id", "page_id")

In [ ]:
# useDeprecated = spark.sql.internal.SQLConf.useDeprecatedKafkaOffsetFetching

In [105]:
query = exploded_df \
    .writeStream \
    .format("console") \
    .option("truncate", "false") \
    .start()

In [107]:
query.awaitTermination()

StreamingQueryException: 'boolean org.apache.spark.sql.internal.SQLConf.useDeprecatedKafkaOffsetFetching()'
=== Streaming Query ===
Identifier: [id = 90693c2b-c7e6-44b1-8ae9-345d5326e1a0, runId = d866f83b-b745-4158-85ea-2eeb24890eb3]
Current Committed Offsets: {}
Current Available Offsets: {}

Current State: INITIALIZING
Thread State: RUNNABLE

In [ ]:
!spark-shell --version

In [ ]:
# from pyspark.sql import SparkSession
# from pyspark.streaming import StreamingContext
# from pyspark.streaming.kafka import KafkaUtils

# # Create a Spark session
# spark = SparkSession.builder.appName("KafkaSparkStreaming").getOrCreate()

# # Create a Streaming Context with a batch interval of 5 seconds
# ssc = StreamingContext(spark.sparkContext, 5)

# # Define Kafka parameters
# kafka_params = {
#     "bootstrap.servers": "localhost:9092",
#     "group.id": "spark-streaming-group",
#     "auto.offset.reset": "latest"
# }
# kafka_topic = "your_kafka_topic_name"

# # Create a DStream that reads from the Kafka topic
# kafka_stream = KafkaUtils.createDirectStream(ssc, [kafka_topic], kafka_params)

# # Process the streaming data from Kafka
# lines = kafka_stream.map(lambda x: x[1])  # Extract message value
# lines.pprint()  # Print the message to the console

# # Start the streaming context
# ssc.start()
# ssc.awaitTermination()

In [ ]:
# from pyspark import SparkContext
# from pyspark.streaming import StreamingContext

# # Create a local StreamingContext with two working thread and batch interval of 1 second
# sc = SparkContext("local[2]", "NetworkWordCount")
# ssc = StreamingContext(sc, 1)

In [ ]:
# lines = ssc.socketTextStream("rc1a-sq626k239t359ent.mdb.yandexcloud.net", 9091)

In [ ]:
# from pyspark.streaming.kafka import KafkaUtils

In [ ]:
# import os
# import sys
# import logging

# os.environ['PYSPARK_PYTHON'] = sys.executable
# os.environ['PYSPARK_DRIVER_PYTHON'] = sys.executable

# import findspark
# findspark.init()
# import pyspark
# from pyspark import SparkContext, SparkConf
# from pyspark.sql import SQLContext, SparkSession
# import pyspark.sql.functions as f
# from pyspark.sql.types import StringType, StructField, StructType, ArrayType, LongType
# from pyspark.sql.types import MapType,StringType
# from pyspark.sql.functions import from_json
# from pyspark.sql.functions import explode, col, split


# def main():
#     spark = (
#         SparkSession
#         .builder
#         .appName("kafka-stream")
#         .config("spark.dynamicAllocation.enabled", "true")
#         .config("spark.executor.cores", "2")
#         .config("spark.executor.memory", "2g")
#         .config("spark.executor.instances", "3")
#         .config("spark.default.parallelism", "24").getOrCreate()
#     )

#     streaming_df = spark.readStream\
#     .format("kafka") \
#     .option("kafka.bootstrap.servers", "rc1a-sq626k239t359ent.mdb.yandexcloud.net:9091") \
#     .option("subscribe", "transactions_sample") \
#     .option("startingOffsets", "earliest") \
#     .load()


#     json_schema = StructType([StructField("ts", StringType(), True), \
#                           StructField("user_id", StringType(), True), \
#                           StructField("page_id", StringType(), True)])

#     json_df = streaming_df.selectExpr("cast(value as string) as value")

#     json_expanded_df = json_df.withColumn("value", from_json(json_df["value"], json_schema)).select("value.*")
#     exploded_df = json_expanded_df.select("ts", "user_id", "page_id")

#     result = exploded_df.writeStream \
#     .format("console") \
#     .option("checkpointLocation","checkpoint_dir") \
#     .outputMode("complete") \
#     .start()

#     result.awaitTermination()

#     spark.stop()
# if __name__ == "__main__":
#     main()
